In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = AutoModel.from_pretrained("", trust_remote_code=True).to(device).float()
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)

model = model.eval()

response, history = model.chat(tokenizer, "你有什么要对我说的吗", history=[])
print(response)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

作为一名人工智能语言模型,我没有实际的情感或意图,也没有自己的思想或意见。我只是根据我所接受到的问题和指令,提供尽可能准确和有用的答案。如果你有任何需要帮助的问题或需要信息,请随时问我,我会尽力回答。


In [14]:
prompt = """中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病则返回原句。

例如:

学生大概做飞机去北京。 -> 学生大概坐飞机去北京。\n
吸烟者反对这样的措施。 -> 吸烟者反对这样的措施。\n

请对下述句子进行修正。返回你修改后的句子，无需其它说明和解释。

xxxxxx ->

"""

def get_prompt(text):
    return prompt.replace('xxxxxx',text)

In [16]:
print(get_prompt('我觉得喜欢一种流行歌曲是盲目的追求一种东西一样。'))
response, his = model.chat(tokenizer, get_prompt('我觉得喜欢一种流行歌曲是盲目的追求一种东西一样。'), history=[])
print(response)  

中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病则返回原句。

下面是一些范例:

学生大概做飞机去北京。 -> 学生大概坐飞机去北京。

我觉得他很大胆，他不去外国。他觉得如果要改转变中国，所以就要从中国做的研究。  -> 我觉得他很大胆，他不去外国。他觉得如果要改变中国，就要在中国做研究。

吸烟者反对这样的措施。 -> 吸烟者反对这样的措施。


请对下述句子进行修正。返回你修改后的句子，无需其它说明和解释。

我觉得喜欢一种流行歌曲是盲目的追求一种东西一样。 ->


我觉得喜欢一种流行歌曲是盲目地追求一种东西。


In [17]:
#增加4个范例
his.append(("因为污染也很高，我们要回收垃圾。为什么?塑料、电池、垃圾是我们做的东西。 -> ","因为污染很严重，所以我们要回收垃圾。为什么?因为塑料、电池等垃圾是我们人类造出来的东西。"))
his.append(("最近吸烟者率是越来越多，而且吸烟者的年龄层是越来越少。这些问题是现在最重要的事。 -> ","最近吸烟率是越来越高，而且吸烟者的年龄层是越来越低。这些问题是现在最重要的事。"))

his.append(("不仅他学习某种习惯，他们性格也开始养成。 -> ","他们不仅开始学习某种习惯，他们的性格也开始形成。"))
his.append(("这世界上是很残酷的。 -> ","这世界是很残酷的。"))
his.append(("每一刻都用得无可挑剔。 -> ","每一刻都用得无可挑剔。"))

In [18]:
response, history = model.chat(tokenizer, "我认为控制青少年的吸烟最重要的是还是家庭教育。 -> ", history=his)
print(response) 

response, history = model.chat(tokenizer, "但是吸烟者还不考虑到被吸烟者。应该政府保护被吸烟者的权利和健康。 -> ", history=his)
print(response) 

response, history = model.chat(tokenizer, "现在，我的生活非常充实，非常满意。 -> ", history=his)
print(response) 

我认为控制青少年的吸烟最重要的是家庭教育。
但是吸烟者却不考虑被吸烟者的权利和健康。政府应该保护被吸烟者的权利和健康。
现在,我的生活非常充实和满意。


**必须要经过这样对模型记忆的调整之后，模型才能稳定地返回答案。**

In [20]:
#封装成一个函数
def predict(text):
    response, history = model.chat(tokenizer, f"{text} ->", history=his,
    temperature=0.01)
    return response 

predict("现在，我的生活非常充实，非常满意。") #可以看到这里的修改就出现错误了。

'现在,我的生活非常充实和满意。'

实用小技巧：编码格式不对：在合适的格式下打开文件，剪切；切换文件编码格式并清空内容，粘贴。

In [42]:
import pandas as pd 
import numpy as np 
import datasets 


df = pd.read_csv("finetune/finetunedataset.csv")
df=df.filter(regex="[原句改句]")
df.head()

/tmp/ipykernel_593456/3949351592.py:6: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("finetune/finetunedataset.csv")


,原句,改句
0,阿阿拉三世和斯特拉托妮可之子。,阿里阿拉特三世和斯特拉托妮可之子。
1,阿巴杜-马汀的父亲是穆斯林，而母亲则昧基督徒。,阿巴杜-马汀的父亲是穆斯林，而母亲则是基督徒。
2,阿巴合在工作中发现，随着村子逐渐赋予，村干部和村民反而有了隔阂。,阿巴合在工作中发现，随着村子逐渐富裕，村干部和村民反而有了隔阂。
3,阿巴斯·亚罗斯塔米也是摄影家与诗人，他的摄影作品（1978年至2003年）大部分是雪景，以在...,阿巴斯·基亚罗斯塔米也是摄影家与诗人，他的摄影作品（1978年至2003年）大部分是雪景，以...
4,阿巴斯下令巴这勒斯坦警察今后停止止这样的攻击。,阿巴斯下令巴勒斯坦警察今后停止这样的攻击。


生成训练集和测试集

In [43]:
ds_dic = datasets.Dataset.from_pandas(df).train_test_split(
    test_size = 2000,shuffle=True, seed = 43)
dftrain = ds_dic['train'].to_pandas()
dftest = ds_dic['test'].to_pandas()
dftrain.to_parquet('finetune/dftrain.parquet')
dftest.to_parquet('finetune/dftest.parquet')

参考官方构建prompt的方法来构建Prompt

In [37]:
def build_inputs(query, history):
    prompt = ""
    for i, (old_query, response) in enumerate(history):
        prompt += "[Round {}]\n\n问：{}\n\n答：{}\n\n".format(i + 1, old_query, response)
    prompt += "[Round {}]\n\n问：{} -> \n\n答：".format(len(history) + 1, query)
    return prompt 
print(build_inputs('味道不太行',history=his))

[Round 1]

问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病则返回原句。

下面是一些范例:

学生大概做飞机去北京。 -> 学生大概坐飞机去北京。

我觉得他很大胆，他不去外国。他觉得如果要改转变中国，所以就要从中国做的研究。  -> 我觉得他很大胆，他不去外国。他觉得如果要改变中国，就要在中国做研究。

吸烟者反对这样的措施。 -> 吸烟者反对这样的措施。


请对下述句子进行修正。返回你修改后的句子，无需其它说明和解释。

我觉得喜欢一种流行歌曲是盲目的追求一种东西一样。 ->



答：我觉得喜欢一种流行歌曲是盲目地追求一种东西。

[Round 2]

问：因为污染也很高，我们要回收垃圾。为什么?塑料、电池、垃圾是我们做的东西。 -> 

答：因为污染很严重，所以我们要回收垃圾。为什么?因为塑料、电池等垃圾是我们人类造出来的东西。

[Round 3]

问：最近吸烟者率是越来越多，而且吸烟者的年龄层是越来越少。这些问题是现在最重要的事。 -> 

答：最近吸烟率是越来越高，而且吸烟者的年龄层是越来越低。这些问题是现在最重要的事。

[Round 4]

问：不仅他学习某种习惯，他们性格也开始养成。 -> 

答：他们不仅开始学习某种习惯，他们的性格也开始形成。

[Round 5]

问：这世界上是很残酷的。 -> 

答：这世界是很残酷的。

[Round 6]

问：每一刻都用得无可挑剔。 -> 

答：每一刻都用得无可挑剔。

[Round 7]

问：味道不太行 -> 

答：


根据模型设置标准化的问答数据集

In [44]:
dftrain['context'] = [build_inputs(x,history=his) for x in dftrain['原句']]
dftrain['target'] = [x for x in dftrain['改句']]
dftrain = dftrain[['context','target']]

dftest['context'] = [build_inputs(x,history=his) for x in dftest['原句']]
dftest['target'] = [x for x in dftest['改句']]
dftest = dftest[['context','target']]

dftest.head()

,context,target
0,[Round 1]\n\n问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病...,按理来说，这种关心应该随着生活水平的提高不断增加才对，现在通过这些单位的执行，反而变成了极大...
1,[Round 1]\n\n问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病...,这些学生往往都具有良好的品质和较高的学习能力，可以通过向优秀榜样学习来提升自我。
2,[Round 1]\n\n问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病...,”【2】士人们的人生观与哲学观自然也随之转变，而其对山水自然美的认识也从以玄对山水转向以佛对...
3,[Round 1]\n\n问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病...,战略成本会计的切入点是尽可能地降低产品成本，加强企业成本优化管理与控制，其关注点有质量控制、...
4,[Round 1]\n\n问：中文语法纠错任务：将一段中文句子进行修正，如果句子没有明显语病...,此外，众多高校在财务管理的内部管理制度方面存在较多纰漏，很多规范过于笼统没有实际可行性，从而...


dataframe处理成模型能够识别的dataset形式

In [45]:
ds_train = datasets.Dataset.from_pandas(dftrain)
ds_val = datasets.Dataset.from_pandas(dftest)
ds_val

Dataset({
    features: ['context', 'target'],
    num_rows: 2000
})

接下来要进行的是模型的微调：模型作为一个语言模型，其核心在于计算词向量产生的概率。所以我们首先需要构建词向量，之后传入模型并且利用梯度下降的算法来降低模型计算的损失，也就是让模型认识到我们提供的这一批数据就是最可靠的数据。然后在之后使用的过程中，我们就可以通过刚才给出的那个特定的提示词让模型再走一遍类似的推理过程，这样理论上就能够得到比较可靠的结果。

In [46]:
from tqdm import tqdm
import transformers

model_name = "chatglm2-6b"
max_seq_length = 512
skip_over_length = True

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True)
#transformer编码器和解码器
config = transformers.AutoConfig.from_pretrained(
    model_name, trust_remote_code=True, device_map='auto')
#用于根据输入的模型名称自动设置模型参数，以便于进行模型的初始化和实例化。
def preprocess(example):#对一条数据进行编码。
    context = example["context"]
    target = example["target"]
    
    context_ids = tokenizer.encode(
            context, 
            max_length=max_seq_length,
            truncation=True)
    
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    
    input_ids = context_ids + target_ids + [config.eos_token_id]#添加一个结束符
    
    return {"input_ids": input_ids, "context_len": len(context_ids),'target_len':len(target_ids)}


OSError: chatglm2-6b is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [47]:
ds_train_token = ds_train.map(preprocess).select_columns(['input_ids', 'context_len','target_len'])
#将函数preprocess迭代地作用与ds_train中的每一个元素，结果返回给ds_train_token
if skip_over_length:
    ds_train_token = ds_train_token.filter(
        lambda example: example["context_len"]<max_seq_length and example["target_len"]<max_seq_length)
        #略过编码长度过大的序列，这里显得有些重复但是我不敢改

NameError: name 'preprocess' is not defined

In [ ]:
ds_val_token = ds_val.map(preprocess).select_columns(['input_ids', 'context_len','target_len'])
if skip_over_length:
    ds_val_token = ds_val_token.filter(
        lambda example: example["context_len"]<max_seq_length and example["target_len"]<max_seq_length)

构建管道

In [ ]:
def data_collator(features: list):
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids) #之后按照batch中最长的input_ids进行padding
    
    input_ids = []
    labels_list = []
    
    for length, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        context_len = feature["context_len"]
        
        labels = (
            [-100] * (context_len - 1) + ids[(context_len - 1) :] + [-100] * (longest - length)
        ) #-100标志位后面会在计算loss时会被忽略不贡献损失，我们集中优化target部分生成的loss
        
        ids = ids + [tokenizer.pad_token_id] * (longest - length)
        
        input_ids.append(torch.LongTensor(ids))
        labels_list.append(torch.LongTensor(labels))
        
        
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

In [ ]:
import torch 
dl_train = torch.utils.data.DataLoader(ds_train_token,num_workers=2,batch_size=4,
                                       pin_memory=True,shuffle=True,
                                       collate_fn = data_collator)
dl_val = torch.utils.data.DataLoader(ds_val_token,num_workers=2,batch_size=4,
                                    pin_memory=True,shuffle=True,
                                     collate_fn = data_collator)


In [ ]:
for batch in dl_train:
    break 

In [ ]:
dl_train.size = 300 #每300个step视作一个epoch，做一次验证

定义模型

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModel.from_pretrained("chatglm2-6b",
                                  load_in_8bit=False, 
                                  trust_remote_code=True, 
                                  device_map='auto')

model.supports_gradient_checkpointing = True  #节约cuda
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
#model.lm_head = CastOutputToFloat(model.lm_head)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True
model.print_trainable_parameters()


训练模型

In [ ]:
from torchkeras import KerasModel 
from accelerate import Accelerator 

class StepRunner:
    def __init__(self, net, loss_fn, accelerator=None, stage = "train", metrics_dict = None, 
                 optimizer = None, lr_scheduler = None
                 ):
        self.net,self.loss_fn,self.metrics_dict,self.stage = net,loss_fn,metrics_dict,stage
        self.optimizer,self.lr_scheduler = optimizer,lr_scheduler
        self.accelerator = accelerator if accelerator is not None else Accelerator() 
        if self.stage=='train':
            self.net.train() 
        else:
            self.net.eval()
    
    def __call__(self, batch):
        
        #loss
        with self.accelerator.autocast():
            loss = self.net(input_ids=batch["input_ids"],labels=batch["labels"]).loss

        #backward()
        if self.optimizer is not None and self.stage=="train":
            self.accelerator.backward(loss)
            if self.accelerator.sync_gradients:
                self.accelerator.clip_grad_norm_(self.net.parameters(), 1.0)
            self.optimizer.step()
            if self.lr_scheduler is not None:
                self.lr_scheduler.step()
            self.optimizer.zero_grad()
            
        all_loss = self.accelerator.gather(loss).sum()
        
        #losses (or plain metrics that can be averaged)
        step_losses = {self.stage+"_loss":all_loss.item()}
        
        #metrics (stateful metrics)
        step_metrics = {}
        
        if self.stage=="train":
            if self.optimizer is not None:
                step_metrics['lr'] = self.optimizer.state_dict()['param_groups'][0]['lr']
            else:
                step_metrics['lr'] = 0.0
        return step_losses,step_metrics
    
KerasModel.StepRunner = StepRunner 


#仅仅保存lora可训练参数
def save_ckpt(self, ckpt_path='checkpoint.pt', accelerator = None):
    unwrap_net = accelerator.unwrap_model(self.net)
    unwrap_net.save_pretrained(ckpt_path)
    
def load_ckpt(self, ckpt_path='checkpoint.pt'):
    self.net = self.net.from_pretrained(self.net,ckpt_path)
    self.from_scratch = False
    
KerasModel.save_ckpt = save_ckpt 
KerasModel.load_ckpt = load_ckpt 


In [ ]:
keras_model = KerasModel(model,loss_fn = None,
        optimizer=torch.optim.AdamW(model.parameters(),lr=2e-6))
ckpt_path = 'waimai_chatglm4'


In [ ]:
keras_model.fit(train_data = dl_train,
                val_data = dl_val,
                epochs=100,patience=5,
                monitor='val_loss',mode='min',
                ckpt_path = ckpt_path,
                mixed_precision='fp16'
               )

验证模型

In [ ]:
from peft import PeftModel 
model = AutoModel.from_pretrained("chatglm2-6b",
                                  load_in_8bit=False, 
                                  trust_remote_code=True, 
                                  device_map='auto')
model = PeftModel.from_pretrained(model,ckpt_path)
model = model.merge_and_unload() #合并lora权重

In [ ]:
def predict(text):
    response, history = model.chat(tokenizer, f"{text} -> ", history=his,
    temperature=0.01)
    return response 

predict("现在，我的生活非常充实，非常满意。") 

In [ ]:
dftest = pd.read_parquet('data/dftest.parquet')

In [ ]:
preds = ['' for x in dftest['text']]

In [ ]:
from tqdm import tqdm 
for i in tqdm(range(len(dftest))):
    text = dftest['text'].loc[i]
    preds[i] = predict(text)

In [ ]:
dftest['pred'] = preds 

In [ ]:
dftest.pivot_table(index='tag',columns = 'pred',values='text',aggfunc='count')

In [ ]:
acc = len(dftest.query('tag==pred'))/len(dftest)
print('acc=',acc)

使用模型&调整模型

In [ ]:
def predict(text,temperature=0.8):
    response, history = model.chat(tokenizer, f"{text} -> ", history=his,
    temperature=temperature)
    return response 

for i in range(10):
    print(predict("现在，我的生活非常充实，非常满意。")) 

: 

保存模型

In [ ]:
model.save_pretrained("chatglm2-6b-waimai", max_shard_size='1GB')

In [ ]:
tokenizer.save_pretrained("chatglm2-6b-CGEC")

In [ ]:
!ls chatglm2-6b 

In [ ]:
!cp  chatglm2-6b/*.py chatglm2-6b-CGEC/

In [ ]:
!ls chatglm2-6b-CGEC

调用模型

In [ ]:
from transformers import  AutoModel,AutoTokenizer
model_name = "chatglm2-6b-waimai" 
tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name,
        trust_remote_code=True).half().cuda()